In [1]:
response = requests.get('https://api.github.com/repos/pytorch/issues',
                        headers={'Accept': 'application/vnd.github.text-match+json'})
print('response code', response.status_code)
print('number of comments',len(response.json()))

NameError: name 'requests' is not defined

In [2]:
from datetime import datetime as dt
import time
import sys
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

sys.setrecursionlimit(10**5)

def handle_rate_limits(response):
    now = dt.now()
    reset_time = dt.fromtimestamp(int(response.headers['X-RateLimit-Reset']))
    remaining_requests = response.headers['X-Ratelimit-Remaining']
    remaining_time =(reset_time-now).total_seconds()
    intervals = remaining_time / (1.0 +int(remaining_requests))
    
    print('Sleeping for', int(intervals))
    time.sleep(intervals)
    return True

def get_all_pages(url, params=None, headers=None):
    output_json=[]
    retry_strategy = Retry(
        total = 5,
        status_forcelist=[500, 503, 504],
        backoff_factor=1
    )

    retry_adapter = HTTPAdapter(max_retries=retry_strategy)

    http = requests.Session()
    http.mount("https://", retry_adapter)
    
    response = http.get(url, params=params, headers=headers)
    if response.status_code == 200:
        output_json = response.json()
        if 'next' in response.links:
            next_url = response.links['next']['url']
            if next_url is not None and handle_rate_limits(response):
                output_json += get_all_pages(next_url, params, headers)
    return output_json

out = get_all_pages(
    "https://api.github.com/repos/pytorch/pytorch/issues/comments",
    params={
        'since': '2022-01-01T10:00:01Z',
        'sorted': 'created',
        'direction': 'desc'
    },
    headers={'Accept' : 'application/vnd.github.v3+json'}
)

df = pd.DataFrame(out)
print(df['body'].count())
df[['id', 'created_at','body']].sample(1)

KeyError: 'body'